In [57]:
import pandas as pd, numpy as np

trip_sheet = '/DATA/TripAnalysis/nextbike_trips.csv'

#before cleaning
trip_df = pd.read_csv(trip_sheet.lstrip('/'))
print(trip_df.head(5).to_string(index=False))
print(f"Length of the dataframe before cleaning: {len(trip_df)}")

trip_df = (
    pd.read_csv(trip_sheet.lstrip('/'))
      .pipe(lambda d: d.rename(columns={c: c.strip() for c in d.columns}))
      .assign(**{
          'Duration-Minutes': lambda d: pd.to_numeric(d['Duration-Minutes'], errors='coerce'),
          'Rental-Lat':       lambda d: pd.to_numeric(d['Rental-Lat'], errors='coerce'),
          'Rental-Lng':       lambda d: pd.to_numeric(d['Rental-Lng'], errors='coerce'),
          'Return-Lat':       lambda d: pd.to_numeric(d['Return-Lat'], errors='coerce'),
          'Return-Lng':       lambda d: pd.to_numeric(d['Return-Lng'], errors='coerce'),
      })
      .loc[lambda d: ~d.apply(lambda r: r.astype(str).str.contains('unbekannt', case=False).any(), axis=1)]
      .loc[lambda d: d['Duration-Minutes'] > 2]
      .loc[lambda d: ~((d['Rental-Lat'] == d['Return-Lat']) & (d['Rental-Lng'] == d['Return-Lng']))]
      .assign(**{
          'Rental-Time': lambda d: pd.to_datetime(d['Rental-Time'], errors='coerce'),
          'weekday_idx': lambda d: d['Rental-Time'].dt.weekday,
          'is_weekend':  lambda d: d['weekday_idx'] >= 5,
          'hour':        lambda d: d['Rental-Time'].dt.hour,
      })
      .reset_index(drop=True)
)

print(trip_df.head(5).to_string(index=False))
print(f"Length of the dataframe after cleaning: {len(trip_df)}")

 Bike-Number         Rental-Time        Rental-Type                                     Rental-Location  Rental-Lat  Rental-Lng         Return-Time        Return-Type                  Return-Location  Return-Lat  Return-Lng  Duration-Minutes   Movement-Type
      100245 2025-07-23 21:56:39 Station (virtuell)                            U Schlesisches Tor (Ost)   52.500794   13.442182 2025-07-23 22:08:17 Station (virtuell)        Elsenstraße/Harzer Straße   52.485136   13.449063              11.6 Station:Station
       13167 2025-07-23 20:22:59 Station (virtuell)                           Elsenstraße/Harzer Straße   52.485136   13.449063 2025-07-23 22:08:17 Station (virtuell) Modersohnstraße (Am Rudolfplatz)   52.502492   13.456503             105.3 Station:Station
       19766 2025-07-23 21:56:31 Station (physisch) Jelbi S+U Zoologischer Garten/Jebensstraße (CHA/JE)   52.506622   13.331480 2025-07-23 22:08:26 Station (virtuell)  Franklinstraße/Glutenbergstraße   52.518675   13.326239   

In [58]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import plotly.graph_objects as go
from plotly.colors import qualitative

# --- 1. Bezirksgrenzen laden ---
berlin = gpd.read_file("./DATA/TripAnalysis/bezirksgrenzen.geojson")

# Spaltenname für Bezirksnamen
district_col = 'Gemeinde_name'

# --- 2. Trip-CSV laden und bereinigen ---
# trip_df sollte bereits geladen und bereinigt sein

# --- 3. Start- und Endpunkte als GeoDataFrames anlegen ---
gdf_start = gpd.GeoDataFrame(
    trip_df,
    geometry=gpd.points_from_xy(trip_df['Rental-Lng'], trip_df['Rental-Lat']),
    crs=berlin.crs
)

gdf_end = gpd.GeoDataFrame(
    trip_df,
    geometry=gpd.points_from_xy(trip_df['Return-Lng'], trip_df['Return-Lat']),
    crs=berlin.crs
)

# --- 4. Spatial Join: Bezirke zuordnen ---
gdf_start = gpd.sjoin(gdf_start, berlin[['geometry', district_col]], how='left', predicate='within')
gdf_start = gdf_start.rename(columns={district_col: 'Start-District'})

gdf_end = gpd.sjoin(gdf_end, berlin[['geometry', district_col]], how='left', predicate='within')
gdf_end = gdf_end.rename(columns={district_col: 'End-District'})

# --- 5. Bezirksnamen ins trip_df übernehmen ---
trip_df['Start-District'] = gdf_start['Start-District']
trip_df['End-District'] = gdf_end['End-District']

# --- 6. Gruppieren nach Bezirks-Paaren ---
district_routes = trip_df.groupby(['Start-District', 'End-District']).size().reset_index(name='count')
district_routes = district_routes.dropna(subset=['Start-District', 'End-District'])
district_routes = district_routes[district_routes['count'] > 10]

# --- 7. Sankey-Diagramm vorbereiten und bauen ---
district_routes['source_node'] = district_routes['Start-District'] + " (Start)"
district_routes['target_node'] = district_routes['End-District'] + " (End)"

start_nodes = sorted(district_routes['source_node'].unique())
end_nodes = sorted(district_routes['target_node'].unique())
all_nodes = start_nodes + end_nodes

node_indices = {name: i for i, name in enumerate(all_nodes)}

unique_starts = sorted(district_routes['Start-District'].unique())
farben = qualitative.Set3
farben_dict = {bezirk: farben[i % len(farben)] for i, bezirk in enumerate(unique_starts)}

# Node-Farben
node_colors = []
for node in all_nodes:
    if node.endswith(' (Start)'):
        bezirk = node.replace(' (Start)', '')
        node_colors.append(farben_dict.get(bezirk, '#cccccc'))
    elif node.endswith(' (End)'):
        bezirk = node.replace(' (End)', '')
        node_colors.append(farben_dict.get(bezirk, '#cccccc'))
    else:
        node_colors.append('#cccccc')

# Link-Farben
link_colors = [farben_dict.get(src.replace(' (Start)', ''), '#cccccc') for src in district_routes['source_node']]

# --- Prozentanteile berechnen ---
total_trips = district_routes['count'].sum()
district_routes['percent'] = district_routes['count'] / total_trips * 100

# Summen je Start- und End-Bezirk (für Knoten-Tooltips)
start_sums = district_routes.groupby('Start-District')['count'].sum().to_dict()
end_sums = district_routes.groupby('End-District')['count'].sum().to_dict()

# --- Tooltips für Knoten (mit Prozent) ---
node_tooltips = []
for node in all_nodes:
    if node.endswith(' (Start)'):
        bezirk = node.replace(' (Start)', '')
        val = start_sums.get(bezirk, 0)
        pct = (val / total_trips * 100) if total_trips > 0 else 0.0
        node_tooltips.append(f"{bezirk} (Start)<br>Fahrten: {val}<br>Anteil: {pct:.2f}%")
    else:
        bezirk = node.replace(' (End)', '')
        val = end_sums.get(bezirk, 0)
        pct = (val / total_trips * 100) if total_trips > 0 else 0.0
        node_tooltips.append(f"{bezirk} (Ende)<br>Fahrten: {val}<br>Anteil: {pct:.2f}%")

# --- Tooltips für Links (mit Prozent) ---
link_tooltips = [
    f"Start: {src}<br>Ende: {tgt}<br>Fahrten: {cnt}<br>Anteil: {pct:.2f}%"
    for src, tgt, cnt, pct in zip(
        district_routes['source_node'],
        district_routes['target_node'],
        district_routes['count'],
        district_routes['percent']
    )
]

# --- Sankey zeichnen ---
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=16,
        label=all_nodes,
        color=node_colors,
        customdata=node_tooltips,
        hovertemplate='%{customdata}<extra></extra>'
    ),
    link=dict(
        source=[node_indices[src] for src in district_routes['source_node']],
        target=[node_indices[tgt] for tgt in district_routes['target_node']],
        value=district_routes['count'],
        color=link_colors,
        customdata=link_tooltips,
        hovertemplate='%{customdata}<extra></extra>'
    )
)])

fig.update_layout(
    font_size=14,
    width=1200,
    height=700,
    # title_text="Trip flows between districts"
)

fig.show()


In [60]:
# Analyse der Movement-Types und Sankey-Diagramm (StartType -> EndType)

import pandas as pd
import plotly.graph_objects as go
from plotly.colors import qualitative

# --- Checks ---
if 'Movement-Type' not in trip_df.columns:
    raise ValueError("Spalte 'Movement-Type' wurde nicht gefunden. Bitte sicherstellen, dass trip_df diese Spalte enthält.")

# --- 1) Movement-Types untersuchen ---
movement = trip_df['Movement-Type'].dropna().astype(str).str.strip()
movement = movement[movement != ""]

# Anzahl und Häufigkeit der Movement-Type-Paare (z.B. 'Station:Station', 'Station:Freefloating', ...)
pair_counts = movement.value_counts().reset_index()
pair_counts.columns = ['Movement-Type', 'count']

print("Anzahl eindeutiger Movement-Types (Paare 'Start:End'):", pair_counts.shape[0])
print("\nTop 10 Movement-Types nach Häufigkeit:")
print(pair_counts.head(10).to_string(index=False))

# --- 2) Start-/End-Typen aus Movement-Type ableiten ---
split = movement.str.split(':', n=1, expand=True)
start_types_series = split[0].str.strip().replace('', 'Unknown')
end_types_series   = split[1].str.strip().fillna('Unknown').replace('', 'Unknown') if split.shape[1] > 1 else pd.Series(['Unknown'] * len(split), index=split.index)

df_types = pd.DataFrame({'StartType': start_types_series, 'EndType': end_types_series})

# Auch die Anzahl eindeutiger Basistypen (über beide Seiten) ausgeben
unique_start_types = sorted(df_types['StartType'].unique())
unique_end_types   = sorted(df_types['EndType'].unique())
unique_types_union = sorted(set(unique_start_types) | set(unique_end_types))

print("\nEindeutige Start-Typen:", len(unique_start_types), unique_start_types)
print("Eindeutige End-Typen:", len(unique_end_types), unique_end_types)
print("Eindeutige Basistypen gesamt (Union):", len(unique_types_union), unique_types_union)

# --- 3) Flows aggregieren ---
flows = (
    df_types
    .groupby(['StartType', 'EndType'])
    .size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
)

if flows.empty:
    raise ValueError("Keine Flows vorhanden (prüfe Daten in 'Movement-Type').")

# --- 4) Sankey vorbereiten ---
start_nodes = sorted(flows['StartType'].unique())
end_nodes   = sorted(flows['EndType'].unique())
all_nodes   = [f"{s} (Start)" for s in start_nodes] + [f"{e} (End)" for e in end_nodes]

node_index = {name: i for i, name in enumerate(all_nodes)}

def hex_to_rgba(hex_color, alpha=1.0):
    if not isinstance(hex_color, str):
        return hex_color
    if hex_color.startswith('rgba(') or hex_color.startswith('rgb('):
        return hex_color
    if not hex_color.startswith('#') or len(hex_color) != 7:
        return hex_color
    h = hex_color.lstrip('#')
    r, g, b = int(h[0:2], 16), int(h[2:4], 16), int(h[4:6], 16)
    return f'rgba({r},{g},{b},{alpha})'

palette = qualitative.Bold
categories = sorted(set(start_nodes + end_nodes))  # nicht genutzt für Mapping; Mapping basiert auf Basistypen unten

base_cats = sorted(set(start_nodes + end_nodes))  # nur zur Vollständigkeit
base_types = sorted(set(start_nodes + end_nodes))  # nicht benötigt

# Farbzuordnung pro Basistyp (ohne Suffix), konsistent für Start/End
base_types_unique = sorted(set(list(flows['StartType'].unique()) + list(flows['EndType'].unique())))
color_map = {bt: palette[i % len(palette)] for i, bt in enumerate(base_types_unique)}

# Node-Farben
node_colors = []
for n in all_nodes:
    if n.endswith(' (Start)'):
        bt = n.replace(' (Start)', '')
        node_colors.append(color_map.get(bt, '#cccccc'))         # Start-Knoten: volle Deckkraft
    else:
        bt = n.replace(' (End)', '')
        node_colors.append(hex_to_rgba(color_map.get(bt, '#cccccc'), 0.8))  # End-Knoten: leicht transparent

# Links
link_sources = [node_index[f"{s} (Start)"] for s in flows['StartType']]
link_targets = [node_index[f"{e} (End)"] for e in flows['EndType']]
link_values  = flows['count'].tolist()
link_colors  = [hex_to_rgba(color_map.get(s, '#cccccc'), 0.45) for s in flows['StartType']]

# Tooltips mit Prozentanteil
total = flows['count'].sum()
flows['percent'] = (flows['count'] / total * 100.0).round(2)

node_start_sums = flows.groupby('StartType')['count'].sum().to_dict()
node_end_sums   = flows.groupby('EndType')['count'].sum().to_dict()

node_tooltips = []
for n in all_nodes:
    if n.endswith(' (Start)'):
        bt = n.replace(' (Start)', '')
        val = node_start_sums.get(bt, 0)
        pct = (val / total * 100.0) if total > 0 else 0.0
        node_tooltips.append(f"{bt} (Start)<br>Fahrten: {val}<br>Anteil: {pct:.2f}%")
    else:
        bt = n.replace(' (End)', '')
        val = node_end_sums.get(bt, 0)
        pct = (val / total * 100.0) if total > 0 else 0.0
        node_tooltips.append(f"{bt} (End)<br>Fahrten: {val}<br>Anteil: {pct:.2f}%")

link_tooltips = [
    f"Start: {s} → End: {e}<br>Fahrten: {c}<br>Anteil: {p:.2f}%"
    for s, e, c, p in zip(flows['StartType'], flows['EndType'], flows['count'], flows['percent'])
]

# --- 5) Sankey zeichnen ---
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=16,
        label=all_nodes,
        color=node_colors,
        customdata=node_tooltips,
        hovertemplate='%{customdata}<extra></extra>'
    ),
    link=dict(
        source=link_sources,
        target=link_targets,
        value=link_values,
        color=link_colors,
        customdata=link_tooltips,
        hovertemplate='%{customdata}<extra></extra>'
    )
)])

fig.update_layout(
    title_text="Flows by Movement Type (Start → End)",
    font_size=14,
    width=1100,
    height=650
)

fig.show()


Anzahl eindeutiger Movement-Types (Paare 'Start:End'): 9

Top 10 Movement-Types nach Häufigkeit:
        Movement-Type  count
      Station:Station 148312
     Flexzone:Station   9777
     Station:Flexzone   6326
   NoFlexzone:Station   1550
   Station:NoFlexzone   1439
    Flexzone:Flexzone    304
NoFlexzone:NoFlexzone     40
  Flexzone:NoFlexzone     27
  NoFlexzone:Flexzone     24

Eindeutige Start-Typen: 3 ['Flexzone', 'NoFlexzone', 'Station']
Eindeutige End-Typen: 3 ['Flexzone', 'NoFlexzone', 'Station']
Eindeutige Basistypen gesamt (Union): 3 ['Flexzone', 'NoFlexzone', 'Station']
